In [ ]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import optimizers
import tensorflow as tf

import pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

seed = 0

### Exploración de datos

In [ ]:
!pip install xlrd==1.2

In [ ]:
df = pd.read_excel('/kaggle/input/air-quality-time-series-data-uci/AirQualityUCI.xlsx')
df = df.rename(columns={'Date': 'date', 
                        'Time': 'time',
                        'CO(GT)': 'co_gt',
                        'NMHC(GT)': 'nmhc_gt',
                        'C6H6(GT)': 'c6h6_gt',
                        'NOx(GT)': 'nox_gt',
                        'NO2(GT)': 'no2_gt',
                        'PT08.S1(CO)': 'co_pt',
                        'PT08.S2(NMHC)': 'nmhc_pt',
                        'PT08.S3(NOx)': 'nox_pt',
                        'PT08.S4(NO2)': 'no2_pt',
                        'PT08.S5(O3)': 'o3_pt',
                        'T': 't',
                        'RH': 'rh',
                        'AH': 'ah',
                       })
df['full_timestamp'] = (df['date'].astype(str) + ' ' + df['time'].astype(str)).map(pd.Timestamp) 
df = df.sort_values('full_timestamp', ascending=True).reset_index(drop=True)
df

Una de las primeras dudas que se pueden plantear es si, efectivamente, hay una medición por hora y si ninguna de esas mediciones tiene valores nulos:

In [ ]:
print('¿La diferencia de tiempo entre dos mediciones es siempre de una hora?:', 
      (df.full_timestamp.diff().iloc[1:] == pd.Timedelta(hours=1)).all())
print('¿Hay valores nulos?', pd.isna(df).any().any())

Vemos que no hay valores nan. Sin embargo, en la descripción del dataset aparece indicado -200 como valor por defecto para los valores faltantes.

In [ ]:
# Fijamos como índice la fecha de cada muestra.
df = df.set_index('full_timestamp')

In [ ]:
# Vamos a ver la proporción de mediciones faltantes de cada variable
display((df.drop(columns=['date', 'time']) == -200).mean())
# y la proporción de horas en las que falta al menos una medición
print('Proporción de horas en las que falta al menos una medición:',
      (df.drop(columns=['date', 'time']) == -200).any(axis=1).mean())
print('Proporción de horas en las que faltan todas las mediciones:',
      (df.drop(columns=['date', 'time']) == -200).all(axis=1).mean())

In [ ]:
# Veamos ahora la proporción de horas en las que falta alguna medición distinta de nmhc_gt, 
# ya que es la que menos aparece:
print('Proporción de horas en las que falta al menos una medición (no nmhc_gt):',
      (df.drop(columns=['date', 'time', 'nmhc_gt']) == -200).any(axis=1).mean())

In [ ]:
# Para facilitar el filtrado de los valores faltantes vamos a sustituirlos por nan.
df = df.replace(to_replace=-200, value=np.nan)

Vamos a ver si existe algún patrón claro en los valores que faltan para ver si es posible rellenarlos de alguna forma.

In [ ]:
for col in df.columns.drop(['date', 'time']):
    fig, ax = plt.subplots(figsize=(10, 10))
    df[df.notna()][col].plot(linewidth=1, label=col)
    plt.scatter(df[df.isna()].index, np.ones(df[df.isna()].shape[0])*(-200), c='r', s=0.5, label='NaN')
    plt.title('Evolución temporal de ({}) y valores NaN'.format(col))
    plt.ylabel(col)
    plt.legend()
    plt.show()

Parece que la densidad de valores NaN es muy alta en todas las variables (no se aprecia ninguna discontinuidad en la línea roja). Veamos si se aprecia algo por horas y por meses.

In [ ]:
for col in df.columns.drop(['date', 'time']):
    fig, ax = plt.subplots(figsize=(10, 10))
    plt.plot(range(24), 
             [df[df[col].isna() & df.index.map(lambda x: x.hour == h)].shape[0] for h in range(24)])
    plt.title('Valores NaN por hora del día ({})'.format(col))
    plt.xticks(ticks=range(24))
    plt.ylabel(col)
    plt.show()

In [ ]:
for col in df.columns.drop(['date', 'time']):
    fig, ax = plt.subplots(figsize=(10, 10))
    plt.plot(range(12), 
             [df[df[col].isna() & df.index.map(lambda x: x.month == m)].shape[0] for m in range(12)])
    plt.title('Valores NaN por mes ({})'.format(col))
    plt.xticks(ticks=range(12))
    plt.ylabel(col)
    plt.show()

Parece haber algún patrón similar en varias variables, pero no parece ser algo claro que nos pueda ayudar a rellenar los datos. La pregunta que habría que hacerse ahora es cómo de grandes son los lapsos temporales en los que no hay ningún valor, para ver si tiene sentido utilizar los valores previos de una variable para rellenar los posteriores.

Vamos a calcular las longitudes de las secuencias de NaN que se encuentran seguidas en los datos.

In [ ]:
for col in df.columns.drop(['date', 'time']):
    s = df.isna().cumsum().diff()[col] 
    mask = s.ne(s.shift())

    ids = s[mask].to_numpy()
    counts = s.groupby(mask.cumsum()).cumcount().add(1).groupby(mask.cumsum()).max().to_numpy()

    ids[ids == 'nan'] = np.nan
    ser_out = pd.Series(counts, index=ids, name='counts')
    ser_out = ser_out[ser_out.index==1]
    
    fig, ax = plt.subplots()
    plt.hist(ser_out)
    plt.title('Histograma de valores NaN seguidos ({})'.format(col))
    plt.show()

Dado que hay muchos casos en los que la secuencia de NaNs es demasiado larga como para asignar a un tiempo t el primer valor anterior a ese tiempo t en el que se tenía la medición, por lo que vamos a probar a rellenar valores de ciertas variables en base a otras que compartan una correlación fuerte.

Vamos a ver la tabla de correlaciones:

In [ ]:
correlations = df.drop(columns=['date', 'time']).corr()
correlations

Parece que va a ser posible hacer la sustitución por este método. Para evitar meter una cantidad excesivo de ruido a los datos, vamos a eliminar las variables t, rh y ah, ya que guardan bastante menos correlación con las demás. 

Para cada variable, vamos a quedarnos en cada caso con las otras dos variables con una correlación más fuerte, para usarlas para rellenar datos.

In [ ]:
df = df.drop(columns=['t', 'ah', 'rh'])
correlations = df.corr()
correlations_map = {col: correlations[col].sort_values()[1:].index.tolist() for col in correlations.columns}
correlations_map

La forma de rellenar datos va a ser:
- Para cada medición en un momento t de una variable v que sea NaN tomaremos, de entre los datos anteriores temporalmente a t, las dos variables disponibles (en ese momento t) con más correlación, v1 y v2, con v.
- Tomaremos las tres mediciones más cercanas a los valores v1 y v2.
- El valor asignado a la variable v será la media de esas tres mediciones.

In [ ]:
def manhattan_distance(p1, p2, q1, q2):
    """
    Distancia manhattan entre dos puntos con dos dimensiones cada uno.
    """
    return np.abs(p1-q1) + np.abs(p2-q2)

In [ ]:
# df_to_fill va a ser el dataframe que vayamos rellenando, para no utilizar como inputs 
# del algoritmo de generación valores creados.
df_to_fill = df.copy()
# Creamos una copia normalizada para obtener una medida de la distancia normalizada.
df_normalized = df.drop(columns=['time', 'date']).copy()
max_norm, min_norm = df_normalized.max(), df_normalized.min()
df_normalized = (df_normalized - min_norm) / (max_norm - min_norm)

for index, row in df_normalized.iterrows():
    for col in df_normalized.columns:
        if pd.isna(row[col]):
            filling_vars = [v for v in correlations_map[col] if pd.notna(row[v])][:2]
            if len(filling_vars) == 2:
                p = row[filling_vars[0]], row[filling_vars[1]]
                aux_df = df_normalized.loc[:index][:-1]
                aux_df = aux_df.loc[pd.notna(aux_df[col])][filling_vars + [col]]
                aux_df['distance'] = aux_df.apply(lambda row: manhattan_distance(row[filling_vars[0]], 
                                                                                 row[filling_vars[1]], 
                                                                                 p[0], p[1]), 
                                                  axis=1)
                new_value = aux_df.nsmallest(3, 'distance')[col].mean()
                df_to_fill.at[index, col] = new_value * (max_norm[col] - min_norm[col]) + min_norm[col]
            else:
                # En caso de no existir ningún valor a partir del cual rellenar, tomaremos el inmediato más cercano.
                df_to_fill.at[index, col] = df_to_fill.loc[:index].iloc[-2][col]

In [ ]:
# df_to_fill.to_csv('df_to_fill.csv', index=False)
# df_to_fill = pd.read_csv('df_to_fill.csv')

### Modelado

Nuestro objetivo va a ser predecir las variables de ground truth (no2_gt, nox_gt, nmhc_gt, co_gt, c6h6_gt) de las próximas 24 horas en base al histórico previo (vamos a limitarnos a las 48 horas anteriores).

In [ ]:
# Longitud de la serie previa a la predicción y de la serie posterior.
previous_series_length = 48
posterior_series_length = 24

# Hacemos un split temporal dejando el 60% para train, un 20% para validación y otro 20% para test.
num_samples = df_to_fill.shape[0]
train_df = df_to_fill.iloc[:int(num_samples*0.6)]
train_df['hour'] = train_df.index.map(lambda x: x.hour)
train_df = train_df.reset_index(drop=True)
val_df = df_to_fill.iloc[int(num_samples*0.6):int(num_samples*0.8)]
val_df['hour'] = val_df.index.map(lambda x: x.hour)
val_df = val_df.reset_index(drop=True)
test_df = df_to_fill.iloc[int(num_samples*0.8):]
test_df['hour'] = test_df.index.map(lambda x: x.hour)
test_df = test_df.reset_index(drop=True)

# Definimos las columnas de variables predictoras y de targets
feature_columns = ['hour', 'co_pt', 'nmhc_pt', 'nox_pt', 'no2_pt', 'o3_pt']
target_columns = ['no2_gt', 'nox_gt', 'nmhc_gt', 'co_gt', 'c6h6_gt']

X_train = train_df[feature_columns + target_columns].values
y_train = train_df[target_columns].values
X_val = val_df[feature_columns + target_columns].values
y_val = val_df[target_columns].values
X_test = test_df[feature_columns + target_columns].values
y_test = test_df[target_columns].values

# Escalamos los datos y guardamos el normalizador para usarlo en las futuras predicciones.
data_scaler = StandardScaler()
target_scaler = StandardScaler()

X_train = data_scaler.fit_transform(X_train)
X_val = data_scaler.transform(X_val)
X_test = data_scaler.transform(X_test)

y_train = target_scaler.fit_transform(y_train)
y_val = target_scaler.transform(y_val)
y_test = target_scaler.transform(y_test)

with open('data_scaler.pkl', 'wb') as f:
    pickle.dump(data_scaler, f)
with open('target_scaler.pkl', 'wb') as f:
    pickle.dump(target_scaler, f)

In [ ]:
def build_timeseries(arr, length=48):
    """
    Devuelve series temporales de un array ordenado temporalmente.
    """
    shape = arr.shape
    ts_arr = np.zeros((shape[0]-length+1, length, shape[1]))
    for n in range(shape[0]-length+1):
        ts_arr[n] = arr[n:length+n]
    return ts_arr

In [ ]:
X_train = build_timeseries(X_train[:-posterior_series_length], length=previous_series_length)
X_val = build_timeseries(X_val[:-posterior_series_length], length=previous_series_length)
X_test = build_timeseries(X_test[:-posterior_series_length], length=previous_series_length)

y_train = build_timeseries(y_train[previous_series_length:], length=posterior_series_length)
y_val = build_timeseries(y_val[previous_series_length:], length=posterior_series_length)
y_test = build_timeseries(y_test[previous_series_length:], length=posterior_series_length)

In [ ]:
X_train, y_train = shuffle(X_train, y_train, random_state=seed)
X_val, y_val = shuffle(X_val, y_val, random_state=seed)
X_test, y_test = shuffle(X_test, y_test, random_state=seed)

Para esta tarea vamos a construir un modelo Seq2Seq basado en redes neuronales de tipo LSTM, especializadas en procesar datos temporales. Esta arquitectura nos va a permitir hacer predicciones de un paso temporal basándonos en las de los pasos previos que ya hemos ido prediciendo.

In [ ]:
def train_seq2seq_hps(latent_dim, optimizer, dropout, recurrent_dropout, lr, epochs=1000, batch_size=64):
    tf.random.set_seed(seed)
    
    encoder_inputs = Input(shape=(None, 11))
    encoder = LSTM(latent_dim, dropout=dropout, recurrent_dropout=recurrent_dropout, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]

    decoder_inputs = Input(shape=(None, 5))
    decoder_lstm = LSTM(latent_dim, dropout=dropout, recurrent_dropout=recurrent_dropout, 
                        return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                         initial_state=encoder_states)
    decoder_dense = Dense(5)
    decoder_outputs = decoder_dense(decoder_outputs)

    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    early_stopping = EarlyStopping(patience=5, min_delta=0.001, restore_best_weights=True)
    opt = optimizer(lr=lr)
    model.compile(optimizer=opt, loss='mse')
    model.fit([X_train, y_train[:,:-1]], y_train[:,1:],
              batch_size=batch_size,
              epochs=epochs,
              validation_data=([X_val, y_val[:,:-1]], y_val[:,1:]),
              callbacks=[early_stopping])

    encoder_model = Model(encoder_inputs, encoder_states)

    decoder_state_input_h = Input(shape=(latent_dim,))
    decoder_state_input_c = Input(shape=(latent_dim,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    encoder_model.save('encoder_{}_{}_{}_{}_{}.h5'.format(latent_dim, 
                                                          opt.__class__.__name__,
                                                          int(dropout*10),
                                                          int(recurrent_dropout*10),
                                                          int(lr*100)))
    decoder_model.save('decoder_{}_{}_{}_{}_{}.h5'.format(latent_dim, 
                                                          opt.__class__.__name__,
                                                          int(dropout*10),
                                                          int(recurrent_dropout*10),
                                                          int(lr*100)))
    
    best_epoch = np.argmin(model.history.history['val_loss'])
    dic = {metric: model.history.history[metric][best_epoch] for metric in ['loss', 'val_loss']}
    dic['latent_dim'] = latent_dim
    dic['optimizer'] = opt.__class__.__name__
    dic['dropout'] = dropout
    dic['recurrent_dropout'] = recurrent_dropout
    dic['lr'] = lr
    
    return dic


hps_list = [{'latent_dim': latent_dim, 
             'optimizer': optimizer, 'dropout': dropout, 'recurrent_dropout': recurrent_dropout, 'lr': lr} 
            for latent_dim in [10, 50, 100]
            for optimizer in [optimizers.Adam, optimizers.RMSprop, optimizers.Adadelta]
            for dropout in [0.0, 0.3]
            for recurrent_dropout in [0.0, 0.3]
            for lr in [0.01, 0.1]]

hps_list = shuffle(hps_list)
results = []
for hps in hps_list:
    print(hps)
    results.append(train_seq2seq_hps(**hps))
    df_results = pd.DataFrame(results)
    df_results.to_csv('results.csv', index=False)
    
def decode_sequence(inputs, encoder_model, decoder_model):
    """
    Recibe un modelo encoder y uno decoder y genera toda la serie temporal a partir de los pasos previos.
    """
    states_value = encoder_model.predict(inputs)
    target_seq = inputs[:,-1:,-5:]
    decoded_sentence = np.zeros((1,posterior_series_length,5))
    
    for i in range(posterior_series_length):
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        decoded_sentence[0,i] = output_tokens[0,0] 
        target_seq = output_tokens
        states_value = [h, c]

    return decoded_sentence


In [ ]:
# Cargamos el dataframe de resultados.
df_results = pd.read_csv('results.csv')

df_results['val_loss'] = df_results.apply(lambda row: row['val_loss'][row['best_epoch']], axis=1)
df_results['loss'] = df_results.apply(lambda row: row['loss'][row['best_epoch']], axis=1)

df_results = df_results.sort_values('val_loss', ascending=True)

In [ ]:
# Cargamos el decoder y el encoder del mejor modelo:
best_comb = df_results.iloc[0]
encoder_model = load_model('encoder_{}_{}_{}_{}_{}.h5'.format(best_comb.latent_dim,
                                                             best_comb.optimizer,
                                                             int(best_comb.dropout*10),
                                                             int(best_comb.recurrent_dropout*10),
                                                             int(best_comb.lr*100)))
decoder_model = load_model('decoder_{}_{}_{}_{}_{}.h5'.format(best_comb.latent_dim,
                                                             best_comb.optimizer,
                                                             int(best_comb.dropout*10),
                                                             int(best_comb.recurrent_dropout*10),
                                                             int(best_comb.lr*100)))
encoder_model.save('best_model_encoder.h5')
decoder_model.save('best_model_decoder.h5')

Vamos a visualizar algunos casos concretos de test para terminar. Lo ideal sería hacerlo en un tramo temporal en el que los datos estén completos. Sin embargo, no es posible porque la variable nmhc_gt no existe en todo el conjunto de test.  Así que vamo a hacer las predicciones directamente en el dataset con los datos rellenados. 

Otra posibilidad hubiera sido eliminarla por completo del dataset.

In [ ]:
fig, ax = plt.subplots(nrows=5, ncols=5, figsize=(25,25))
for n in range(5):
    previous_ts = data_scaler.inverse_transform(X_test[n])[:,6:]
    future_ts = target_scaler.inverse_transform(y_test[n])
    total_ts = np.concatenate([previous_ts, future_ts])
    pred = target_scaler.inverse_transform(decode_sequence(X_test[n:n+1], encoder_model, decoder_model))[0]
    for i in range(5):
        ax[n, i].plot(range(72), total_ts[:,i], label=['Ground Truth'])
        ax[n, i].vlines(48, min(total_ts[:,i].min(), pred[:,i].min()), max(total_ts[:,i].max(), pred[:,i].max()))
        ax[n, i].plot(range(48, 72), pred[:,i], color='r', label=['Predicted'])
        ax[n, i].set_title(target_columns[i])
        ax[n, i].set_ylabel(target_columns[i])
        ax[n, i].set_xlabel('timestep')
        ax[n, i].set_xticks(range(0, 72, 12))
        
handles, labels = ax[0,0].get_legend_handles_labels()
fig.legend(handles, list(map(lambda x: x[2:-2], labels)), loc='upper center', prop={'size': 20})
plt.show()

### Conclusión

El resultado parece ser bastante bueno a simple vista, además de coherente, pues todas las variables evolucionan de forma similar. Faltaría hacer el cálculo del error por cada timestep, que probablemente vaya incrementándose ligeramente a lo largo de la serie predicha.

Debido a falta de tiempo, no se ha podido realizar una búsqueda de hiperparámetros más exhaustiva, lo que hubiera dado unos resultados más precisos.

También quedaría pendiente comparar el rendimiento del seq2seq con otros modelos, así como probar otros métodos de relleno de valores NaN.

In [ ]:
# Vamos a generar un df de prueba para lanzar el script.
df = pd.read_excel('AirQualityUCI.xlsx')
test_df = df.fillna(0)[:48]
test_df.to_csv('test_df.csv', index=False)